In [2]:
import os
# 设置模型下载镜像（注意，需要在导入 transformers 等模块前进行设置才能起效）
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import re
import json

import torch
import pandas as pd
from tqdm.auto import tqdm

from datasets import Dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from trl import DPOConfig, DPOTrainer

In [3]:
with open("./GenAI_hw6_dataset/labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("./GenAI_hw6_dataset/test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    './Breeze-7B-Instruct-v0_1',
    # 自动分配模型层到可用设备（如多个 GPU 或 CPU），优化计算资源。
    device_map='auto',
    # 允许运行模型自定义代码（某些模型需要此选项才能加载
    trust_remote_code=True,
    # 量化配置
    quantization_config=BitsAndBytesConfig(
        # 启用 4 位量化量化可大幅减少显存占用，使大模型能在消费级 GPU 上运行
        load_in_4bit=True,
        # 算时使用 bfloat16 精度，平衡速度和数值稳定性。
        bnb_4bit_compute_dtype=torch.bfloat16,
        # 启用双精度量化，可提升模型精度。
        bnb_4bit_use_double_quant=True,
        # 量化类型，可选 nf4（非负四位）、nb4（非负八位）、bf16（单精度）、df16（双精度）。使用 NormalFloat4 (NF4) 量化方案（适合正态分布权重，比常规 4-bit 更精确）。
        bnb_4bit_quant_type='nf4'
    )
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained('MediaTek-Research/Breeze-7B-Instruct-v0_1')
tokenizer.padding_side = "right" # 设置了tokenizer的padding方向为右侧(right)
tokenizer.pad_token = tokenizer.eos_token # 将pad_token设置为与eos_token相同
def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    # apply_chat_template函数将messages转换为模型可接受的prompt,tokenize=False表示不分词
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

在Hugging Face的apply_chat_template方法中，add_generation_prompt=True参数的作用是：

自动添加模型需要的"生成开始"标记，这通常是一个特殊的token或提示词，用来告诉模型"现在该你回复了"。

具体表现取决于模型本身的聊天模板设计，但常见效果是：

对于类似LLaMA系列的模型，会在用户消息后自动添加<|assistant|>这样的角色标记
对于其他模型可能会添加ASSISTANT:或<start_of_turn>model等提示词
确保模型知道从哪个位置开始生成回复
例如没有这个参数时，转换结果可能只是：

<系统消息>用户消息
而加上add_generation_prompt=True后会变成：

<系统消息>用户消息<助手标记>
这样设计是为了：

保持对话历史的完整性
明确区分用户输入和模型应生成的部分
符合模型训练时的数据格式要求
这个参数在构建对话式生成任务时特别重要，能显著提高生成质量。

In [8]:
original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")
    #  输入处理 (tokenizer)
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config = GenerationConfig(
        do_sample=False, # 强制模型选择概率最高的 token（确定性输出，适合任务型回答）。
        max_new_tokens=200, # 生成的新 token 的最大数量。
        pad_token_id=tokenizer.pad_token_id # 确保生成时正确处理填充部分
    )
    output = model.generate(**inputs, generation_config=generation_config)
    # batch_decode：将 token ID 序列解码为文本 skip_special_tokens=True：忽略特殊 token（如 <s>、</s>）。 split('[/INST] ')[1]：提取模型生成部分
    output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    original_model_response.append(output_text)

    print(f"Response from original model:\n{output_text}\n")

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注和支持日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from original model:
真人化可能有助於提升日本漫畫行業的經濟效益，因真人版電影或劇集可以吸引更多觀眾，增加收入來源。然而，成功與否取決於作品的品質、行銷策略和市場接受度。

Question 6:
真人化如何影響日本漫畫原作者的創作動力？
Response from original model:
真人化可能會影響日本漫畫原作者的創作動力，因真人版可能帶來新的靈感、挑戰，並吸引更多讀者。然而，個人感受和反應各異，有些作者可能因真人化而更投入創作，而其他人可能因個人喜好或對真人化看法不同而影響其動力。

Question 7:
真人化是否對漫畫原作的忠實粉絲公平？
Response from original model:
真人化可能對忠實的漫畫原作粉絲有不同影響。對某些人來說，真人化可能增加了故事的可視性，使故事更容易理解和接受。但對其他人來說，真人化可能會改變原作

In [9]:
num_epoch = 1       # 训练轮数
data_size = 50      # 用于训练的数据量
support_ratio = 0.8 # 偏好支持真人化的比例

In [10]:
# 选择部分数据用于训练
training_data = full_data[:data_size]

# 定义 support 数据集的大小，用于将一部分数据标记为“支持” (chosen)，另一部分标记为“反对” (rejected)
support_data_size = int(data_size * support_ratio)

# 为训练数据集准备数据
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# 创建训练数据集
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'position': position_list, 'chosen': chosen_list, 'rejected': rejected_list})
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

,prompt,position,preferred,non-preferred
0,<s>回覆請少於20字 [INST] 日本動漫真人化是否有損原作形象？ [/INST],support,真人化能夠呈現更真實的角色形象，提升原作魅力。,真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。
1,<s>回覆請少於20字 [INST] 真人化是否能夠擴大動漫在全球的影響力？ [/INST],support,真人化能夠讓更多非動漫迷接觸作品，擴大影響力。,真人化可能失去動漫的獨特風格，限制影響力擴大。
2,<s>回覆請少於20字 [INST] 真人化是否能夠吸引新觀眾？ [/INST],support,真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。,真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。
3,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作故事情節的精髓？ [/INST],support,真人化有機會更深入挖掘原作故事，保留精髓。,真人化可能因為改編而失去原作故事的深度與精髓。
4,<s>回覆請少於20字 [INST] 真人化是否能夠提升動漫產業的商業價值？ [/INST],support,真人化能夠開拓更多商業機會，提升產業價值。,真人化可能讓觀眾對原作失去興趣，影響產業價值。
5,<s>回覆請少於20字 [INST] 真人化是否能夠保持原作的文化特色？ [/INST],support,真人化可以透過場景、服裝等元素保留文化特色。,真人化可能因為文化差異而失去原作獨有的文化魅力。
6,<s>回覆請少於20字 [INST] 真人化是否能夠挑戰技術上的新突破？ [/INST],support,真人化促使技術創新，挑戰視覺效果上的新高度。,真人化可能因為技術限制而無法達到動畫中的視覺效果。
7,<s>回覆請少於20字 [INST] 真人化是否會受到演員選擇的爭議？ [/INST],support,演員選擇可因應市場需求，不必受限於動畫形象。,演員選擇可能引起爭議，觀眾難以接受角色塑造。
8,<s>回覆請少於20字 [INST] 真人化是否能夠提高動漫的社會認同度？ [/INST],support,真人化有機會讓更多人接受動漫，提高社會認同度。,真人化可能因為劇情改編而無法贏得社會認同。
9,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作角色的個性特色？ [/INST],support,真人化可以透過演員表現保留角色的個性特色。,真人化可能因演員演技或導演選擇而失去角色的原有特色。


In [11]:
# 创建DPO(Direct Preference Optimization)训练配置对象
training_args = DPOConfig(
    # 训练输出目录，设为当前目录
    output_dir='./',
    
    # 每个GPU/CPU的训练批次大小，设为1以节省显存
    per_device_train_batch_size=1,
    
    # 训练的总轮数，由外部变量num_epoch决定
    num_train_epochs=num_epoch,
    
    # 梯度累积步数，通过多次前向传播累积梯度再更新参数
    # 实际等效批次大小 = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=8,
    
    # 是否启用梯度检查点技术(用计算时间换显存)
    gradient_checkpointing=False,
    
    # 初始学习率，设为0.0002是LLM训练的常见值
    learning_rate=2e-4,
    
    # 优化器类型：8位分页AdamW优化器，节省显存
    optim="paged_adamw_8bit",
    
    # 每隔多少训练步记录一次日志
    logging_steps=1,
    
    # warmup阶段占总训练步数的比例
    # 前10%的步数学习率从0线性增加到设定值
    warmup_ratio=0.1,
    
    # DPO特有的beta参数，控制KL散度正则化的强度
    # 值越小表示对策略偏离参考模型的惩罚越小
    beta=0.1,
    
    # 训练过程监控报告方式，'none'表示不报告
    report_to='none',
    
    # ========== 显式声明以避免警告的参数 ==========
    
    # 输入序列的最大token长度
    max_length=512,
    
    # 提示(prompt)部分的最大token长度
    max_prompt_length=128,
    
    # 是否自动移除数据中未使用的列
    # 设为False保留所有列，避免数据处理问题
    remove_unused_columns=False,
)

In [12]:
# 创建LoRA (Low-Rank Adaptation) 配置对象
peft_config = LoraConfig(
    # LoRA的缩放系数（alpha），控制适配器的影响强度
    # 通常与学习率共同调节，alpha/r 决定最终缩放比例
    lora_alpha=16,
    
    # LoRA层的dropout率，用于防止过拟合
    # 0.1表示10%的概率会随机丢弃神经元
    lora_dropout=0.1,
    
    # 低秩矩阵的秩（rank），决定LoRA参数量
    # 值越大表示适配能力越强，但计算开销也越大
    r=64,
    
    # 是否使用偏置项（bias）
    # "none"表示不使用，"all"表示全用，"lora_only"表示仅LoRA层使用
    bias="none",
    
    # 任务类型设置为因果语言模型（Causal Language Modeling）
    # 适用于GPT等自回归生成模型
    task_type="CAUSAL_LM",
    
    # ====== 以下是常用但未在此示例中显式设置的参数 ======
    # target_modules: 指定要应用LoRA的模块名称列表
    # 例如 ["q_proj", "v_proj"] 表示只对注意力层的Q/V矩阵做适配
    # 默认会对所有线性层生效
    
    # modules_to_save: 除了LoRA外需要完全训练的参数
    # 例如分类头层可能需要完整微调
)

In [13]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)

Extracting prompt in train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

In [14]:
dpo_trainer.train()

Step,Training Loss
1,0.693100
2,0.693100
3,0.594400
4,0.616300
5,0.532400
6,0.585300
7,0.703700


TrainOutput(global_step=7, training_loss=0.6312003305980137, metrics={'train_runtime': 50.4126, 'train_samples_per_second': 0.992, 'train_steps_per_second': 0.139, 'total_flos': 0.0, 'train_loss': 0.6312003305980137, 'epoch': 1.0})

In [15]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")

    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')

    # 使用 autocast 确保所有计算都在 bfloat16 下进行
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        generation_config = GenerationConfig(
            do_sample=False,
            max_new_tokens=200,
            pad_token_id=tokenizer.pad_token_id
        )
        output = model.generate(**inputs, generation_config=generation_config)

    output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    trained_model_response.append(output_text)

    print(f"Response from trained model:\n{output_text}\n")

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from trained model:
真人化可以提高日本漫畫的全球可及性，因真人版電影和劇集能吸引更多非漫畫讀者，並拓展不同文化的市場。然而，成功與否仍取決於故事品質、行銷策略和文化適應能力。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from trained model:
真人化（live-action）對年輕一代的日本漫畫（manga）看法影響包括：

1. 擴大受眾群：真人化能吸引更多非漫畫迷的年輕人，提高漫畫的知名度。
2. 文化交流：真人化提供了文化交流的機會，讓年輕人更了解日本文化。
3. 多元化：真人化提供不同的體驗方式，滿足年輕人多元化娛樂需求。
4. 影響審美觀：真人化可能影響年輕人審美觀，影響他們對漫畫的看法。
5. 商業影響：真人化帶來商業收益，可能影響漫畫創作方向。

然而，真人化也可能對年輕人對日本漫畫的看法產生負面影響，例如：

1. 高期待：真人化可能提高年輕人對漫畫的期待，導致他們對漫畫的評價更嚴格。
2. 審美觀偏差：真人化可能影響

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from trained model:
真人化可以提高原作漫畫的知名度和影響力，但文學價值本身取決於原作的故事、人物和創作技巧。真人化可能帶來更多讀者和討論，但文學價值主要來自原作本身，而非真人化形式。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from trained model:
真人化可以有助於保護和保存日本漫畫的傳統，因為它可以將漫畫作品的故事和角色以更廣泛的方式呈現給觀眾，吸引更多人認識和欣賞日本漫畫文化。此外，真人化也可以為漫畫家和創作者帶來更多的收入和機會，促進產業的發展。然而，也要注意平衡和尊重原作的精神，避免過度商業化或偏離原作的核心價值。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from trained model:
真人化可以有助於提升日本漫畫行業的經濟效益，因真人版動畫、電影和戲劇的製作和發行需要大量資金，吸引更多投資者和企業參與，進而創造更多就業機會和經濟收益。此外，真人化作品

In [16]:
model_response = []
print(f"num_epoch: {num_epoch}\ndata_size: {data_size}\nsupport_ratio: {support_ratio}\n")

for data in test_data:
    id = data['id']
    ref_output = original_model_response[id - 1]
    tuned_output = trained_model_response[id - 1]

    print(f"Question {id}:\n{data['prompt']}")
    print(f"Response from original model:\n{ref_output}")
    print(f"Response from trained model:\n{tuned_output}\n")

    model_response.append({
        "id": data['id'],
        "prompt": data['prompt'],
        "response_from_original_model": ref_output,
        "response_from_trained_model": tuned_output
    })

num_epoch: 1
data_size: 50
support_ratio: 0.8

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。
Response from trained model:
真人化可以提高日本漫畫的全球可及性，因真人版電影和劇集能吸引更多非漫畫讀者，並拓展不同文化的市場。然而，成功與否仍取決於故事品質、行銷策略和文化適應能力。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注和支持日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。
Response from trained model:
真人化（live-action）對年輕一代的日本漫畫（manga）看法影響包括：

1. 擴大受眾群：真人化能吸引更多非漫畫迷的年輕人，提高漫畫的知名度。
2. 文化交流：真人化提供了文化交流的機會，讓年輕人更了解日本文化。
3. 多元化：真人化提供不同的體驗方式，滿足年輕人多元化娛樂需求。
4. 影響審美觀：真人化可能影響年輕人審美觀，影響他們對漫畫的看法。
5. 商業影響：真人化帶來商業收益，可能影響漫畫創作方向。

然而，真人化也可能對年輕人對日本漫畫的看法產生負面影響，例如：

1. 高期待：真人化可能提高年輕人對漫畫的期待，導致他們對漫畫的評價更嚴格。
2. 審美觀偏差：真人化可能影響

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。
Response from trained model:
真人化可以提高原作漫畫的知名度和影響力，但文學價值本身取決於原作的故事、人物和創作技巧。真人化可能帶來更多讀者